### Imports and paths

In [136]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import PIL
from IPython.display import display, clear_output
import time
import math
import warnings

FilePath = "IMG_DATA"
Bands = {
    "SLO":{
        "2" : {
            "path": FilePath+"_SLO/" + "T33TUL_20220125T100311_B02.jp2",
            "res" : 10,
        },
        "3" : {
            "path": FilePath+"_SLO/" + "T33TUL_20220125T100311_B03.jp2",
            "res" : 10,
        },
        "4" : {
            "path": FilePath+"_SLO/" + "T33TUL_20220125T100311_B04.jp2",
            "res" : 10,
        },
        "8" : {
            "path": FilePath+"_SLO/" + "T33TUL_20220125T100311_B08.jp2",
            "res" : 10,
        },
        "11" : {
            "path": FilePath+"_SLO/" + "T33TUL_20220125T100311_B11.jp2",
            "res" : 20,
        },
    },
    "BRA":{
        "2" : {
            "path": FilePath+"_BRA/" + "T22LER_20220125T134211_B02.jp2",
            "res" : 10,
        },
        "3" : {
            "path": FilePath+"_BRA/" + "T22LER_20220125T134211_B03.jp2",
            "res" : 10,
        },
        "4" : {
            "path": FilePath+"_BRA/" + "T22LER_20220125T134211_B04.jp2",
            "res" : 10,
        },
        "8" : {
            "path": FilePath+"_BRA/" + "T22LER_20220125T134211_B08.jp2",
            "res" : 10,
        },
        "11" : {
            "path": FilePath+"_BRA/" + "T22LER_20220125T134211_B11.jp2",
            "res" : 20,
        },
    },
    "SWE":{
        "2" : {
            "path": FilePath+"_SWE/" + "T33VWF_20220121T102331_B02.jp2",
            "res" : 10,
        },
        "3" : {
            "path": FilePath+"_SWE/" + "T33VWF_20220121T102331_B03.jp2",
            "res" : 10,
        },
        "4" : {
            "path": FilePath+"_SWE/" + "T33VWF_20220121T102331_B04.jp2",
            "res" : 10,
        },
        "8" : {
            "path": FilePath+"_SWE/" + "T33VWF_20220121T102331_B08.jp2",
            "res" : 10,
        },
        "11" : {
            "path": FilePath+"_SWE/" + "T33VWF_20220121T102331_B11.jp2",
            "res" : 20,
        },
    },
}

### Helper and main functions

In [137]:
def read_jp_2(band, region):
    filename = Bands[region][band]["path"]
    raw = rasterio.open(filename).read(1)
    # raw = PIL.Image.open(filename)
    arr = np.array(raw)
    return arr


def resize(arr, times):
    arr = arr.repeat(times, axis=1)
    arr = arr.repeat(times, axis=0)
    return arr

def EVI(region):
    b8 = read_jp_2("8", region)
    b4 = read_jp_2("4", region)
    b2 = read_jp_2("2", region)
    new_index = (b8 - b4) / (b8 + (6.0*b4) - (7.5*b2) + 1.0)
    return new_index

def NDWI(region):
    b11 = read_jp_2("11",region)#res 20
    b3 = read_jp_2("3",region)#res 10
    b11 = resize(b11,2)
    new_index = (b3 - b11)/(b3 + b11 + 0.01)
    return new_index

def save(npArr, limits, mode, name):
    print("Saving as "+ name + ".tiff" )
    plt.imsave(name + '.tiff', npArr,vmin=limits[0],vmax=limits[1], cmap=mode)

def show_np(npArr, limits=None, mode=None):
    plt.imshow(npArr, clim=limits, cmap=mode)
    plt.show()

def get_win_size(S):
    return 2*S + 1

def get_pad_size(window):
    return math.floor(window/2)

def expand_to_pad(img, pad):
    new = np.zeros((img.shape[0] + 2*pad, img.shape[1] + 2*pad))
    new[pad:img.shape[0]+pad, pad:img.shape[1]+pad] = img
    return new

def stride_img(inputimage,S,function):
    win = get_win_size(S)
    pad = get_pad_size(win)
    windowed = expand_to_pad(inputimage,pad)
    windowed = np.lib.stride_tricks.sliding_window_view(windowed,(win,win))
    windowed = windowed.reshape(windowed.shape[0],windowed.shape[1],windowed.shape[2]*windowed.shape[3])

    if function == 'expand':
        mod_img = shrink_expand(windowed, 'expand')
    elif function == 'shrink':
        mod_img = shrink_expand(windowed, 'shrink')
    else:
        raise Exception("Function not recognized")
    return mod_img

def shrink_expand(img, method='shrink'):
    if method == 'shrink':
        ret = np.amin(img,axis=2)
        return ret
    elif method == 'expand':
        ret = np.amax(img,axis=2)
        return ret
    else:
        raise Exception("Method not recognized")

def reconstruction_check(img, val, method='open'):
    if method == 'open':
        ret = np.minimum(img,val)
        return ret
    elif method == 'close':
        ret = np.maximum(img,val)
        return ret
    else:
        raise Exception("Method not recognized")


def OC(img, S, method='open'):
    if method == 'open':
        img = stride_img(img,S,'shrink')
        img = stride_img(img,S,'expand')
        return img
    elif method == 'close':
        img = stride_img(img,S,'expand')
        img = stride_img(img,S,'shrink')
        return img
    else:
        raise Exception("Method not recognized")


def oc_reconstruction(inp, S, method='open', fliter_shape='square'):
    start = time.time()
    print('Reconstruction - ' + method)
    counter = 0
    if method == 'open':
        markImg = stride_img(inp,S,'shrink')
    elif method == 'close':
        markImg = stride_img(inp,S,'expand')
    else:
        raise Exception("Method not recognized")
    while True:
        if method == 'open':
            vrednost = stride_img(markImg,1,'expand')
            img = reconstruction_check(inp,vrednost,'open')
        else:
            vrednost = stride_img(markImg,1,'shrink')
            img = reconstruction_check(inp,vrednost,'close')
        if(np.array_equal(markImg,img)):
            break
        markImg = img
        counter += 1
        clear_output()
    end = time.time()
    print('Done. Iterations: ' + str(counter) + ' Time: ' + str(end-start))
    return img


### Main processing

In [138]:
S = 4

print('Region: SLO')
SLO = NDWI("SLO")
SLO = SLO[0:1000, -1000:]
opened = OC(SLO,S, 'open')
closed = OC(SLO,S, 'close')
open_reconstsruction = oc_reconstruction(SLO,S, 'open')
close_reconstruction = oc_reconstruction(SLO,S, 'close')
plt.imsave('output/SLO.png', SLO)
plt.imsave('output/SLO_opened.png', opened)
plt.imsave('output/SLO_closed.png', closed)
plt.imsave('output/SLO_opened_reconstruction.png', open_reconstsruction)
plt.imsave('output/SLO_closed_reconstruction.png', close_reconstruction)


print('Region: SWE')
SWE = NDWI("SWE")
plt.figure()
plt.imshow(SWE)
plt.show()
SWE = SWE[9000:10000, 3000:4000]
opened = OC(SWE,S, 'open')
closed = OC(SWE,S, 'close')
open_reconstsruction = oc_reconstruction(SWE,S, 'open')
close_reconstruction = oc_reconstruction(SWE,S, 'close')
plt.imsave('output/SWE.png', SWE)
plt.imsave('output/SWE_opened.png', opened)
plt.imsave('output/SWE_closed.png', closed)
plt.imsave('output/SWE_opened_reconstruction.png', open_reconstsruction)
plt.imsave('output/SWE_closed_reconstruction.png', close_reconstruction)

print('Region: BRA')
BRA = NDWI("BRA")
BRA = BRA[0:1000, -1000:]
opened = OC(BRA,S, 'open')
closed = OC(BRA,S, 'close')
open_reconstsruction = oc_reconstruction(BRA,S, 'open')
close_reconstruction = oc_reconstruction(BRA,S, 'close')
plt.imsave('output/BRA.png', BRA)
plt.imsave('output/BRA_opened.png', opened)
plt.imsave('output/BRA_closed.png', closed)
plt.imsave('output/BRA_opened_reconstruction.png', open_reconstsruction)
plt.imsave('output/BRA_closed_reconstruction.png', close_reconstruction)

Done. Iterations: 141 Time: 12.637478828430176
